# Vector DB 구축 
1. 데이터 확인 - 텍스트,자연어,테이블데이터 혼합형 / 코드확인 및 수동으로 확인 
2. 데이터 전처리 및 기본 청크 진행 
3. 청크 확인 
4. 청크 보완작업 ( 슬라이딩 윈도우 방식 ) 
5. 임베딩 (kure-v1 / faiss db) 
6. 질문지 생성


In [1]:
# 1. 데이터 확인

from docx import Document
from pathlib import Path

# 📂 파일 경로 설정
file_path = Path("work1/data/DM_rules.docx").resolve()

# 📄 문서 불러오기
doc = Document(file_path)

# 📑 전체 문단 확인
print(f"✅ 총 문단 수: {len(doc.paragraphs)}")

# 🔍 앞부분 30개 문단 미리보기
for i, para in enumerate(doc.paragraphs[:30]):
    print(f"{i:02d}: '{para.text.strip()}'")


✅ 총 문단 수: 223
00: 'DM_solution 사내규정'
01: ''
02: '목차'
03: '출장 및 여비규정'
04: '퇴직금 지급 및 퇴직 규정'
05: '복리후생 규정'
06: '복무 규정'
07: '급여 규정'
08: '업무 규정'
09: '정보보안 규정)'
10: '윤리 및 행동강령'
11: '사내 여비규정 (출장규정)'
12: '제1장 총 칙'
13: '제1조【목적】
이 규정은 직원의 국내 및 해외 출장에 따른 여비 지급 및 처리 기준을 정함으로써, 경비의 합리적 지출과 출장업무의 원활한 수행을 도모하는 것을 목적으로 한다.'
14: '제2조【적용범위】
이 규정은 본사 및 국내·해외 지사 소속 전 직원에게 적용한다. 단, 계약직 및 인턴사원은 별도의 승인 없이 출장할 수 없다.'
15: '제3조【정의】
이 규정에서 “출장”이라 함은 회사의 업무를 수행하기 위하여 본래 근무지를 떠나 일시적으로 다른 지역에 체류하는 것을 말한다.'
16: ''
17: '제2장 출장 승인 및 신청'
18: '제4조【출장 승인】
모든 출장은 사전에 소속 부서장과 경영지원팀의 승인을 받아야 하며, 승인된 출장만 여비가 지급된다.'
19: '제5조【출장 신청 절차】
① 출장신청서는 출장 3일 전까지 시스템에 등록해야 한다.
② 해외출장의 경우, 별도의 사전보고서와 현지 연락처가 포함된 계획서를 첨부해야 한다.'
20: ''
21: '제3장 여비의 종류 및 기준'
22: '제6조【여비 구성 항목】
여비는 다음 각 항목으로 구성한다.'
23: '교통비'
24: '일비(숙박 외 체재비)'
25: '숙박비'
26: '식비'
27: '통신비 및 잡비(필요 시 인정)'
28: '제7조【교통비 지급 기준】
① 국내 출장 시에는 대중교통을 원칙으로 한다.
② 자가용 사용 시 사전 승인과 주행거리 증빙자료(네비게이션 캡처 등)를 첨부해야 한다.
③ 해외출장 시에는 이코노미 클래스만 이용가능하다. 사업 목적의 동반자가 있을 경우에는 퍼스트 클래스 이용이 가능하다. ( CEO포함 전 

In [13]:
# 2. 데이터 전처리 및 기본 청크 진행

from docx import Document
import json
import re
from pathlib import Path
from uuid import uuid4

# 1. 경로 설정
docx_path = Path("work1/data/DM_rules.docx")  # 실제 전체 문서
output_path = Path("work1/data/dm_chunks.json")

# 2. 문서 로드
doc = Document(docx_path)

# 3. 문단 + 표 포함 전체 텍스트 수집
elements = []

# 문단 추가
for para in doc.paragraphs:
    text = para.text.strip()
    if text:
        elements.append(text)

# 표 추출
for table in doc.tables:
    table_text = []
    for row in table.rows:
        cells = [cell.text.strip() for cell in row.cells]
        table_text.append(" | ".join(cells))
    elements.append("\n".join(table_text))

# 4. 조문 기준 전처리
chunks = []
current_title = ""
current_body = []

for line in elements:
    if re.match(r"^제\d+조[【\[ ].*?[】\]]", line):  # 새로운 조문 시작
        if current_title and current_body:
            chunks.append({
                "text": "\n".join(current_body),
                "metadata": {
                    "section": current_title,
                    "chunk_id": f"{current_title}_{str(uuid4())[:8]}"
                }
            })
        current_title = line
        current_body = []
    else:
        current_body.append(line)

# 마지막 청크 추가
if current_title and current_body:
    chunks.append({
        "text": "\n".join(current_body),
        "metadata": {
            "section": current_title,
            "chunk_id": f"{current_title}_{str(uuid4())[:8]}"
        }
    })

# 5. 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

# 6. 통계 출력
print(f"✅ 총 요소 수 (문단+표 포함): {len(elements)}")
print(f"✅ 생성된 청크 수: {len(chunks)}")
print(f"📁 저장 완료: {output_path.name}")


✅ 총 요소 수 (문단+표 포함): 182
✅ 생성된 청크 수: 42
📁 저장 완료: dm_chunks.json


In [15]:
# 3. 청크 확인 

import json
import re
from collections import Counter

# 파일 로드
with open("work1/data/dm_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# 중복 ID 체크
ids = [chunk["metadata"]["chunk_id"] for chunk in chunks]
dup_ids = [item for item, count in Counter(ids).items() if count > 1]

# 길이 통계
lengths = [len(chunk["text"]) for chunk in chunks]
avg_len = sum(lengths) // len(lengths)
min_len = min(lengths)
max_len = max(lengths)

# 조문 형식 체크
pattern = r"제\d+조[【\[ ].*?[】\]]"
valid_titles = [chunk["metadata"]["section"] for chunk in chunks if re.search(pattern, chunk["metadata"]["section"])]
invalid_titles = [chunk["metadata"]["section"] for chunk in chunks if not re.search(pattern, chunk["metadata"]["section"])]

# 출력
print(f"✅ 총 청크 수: {len(chunks)}")
print(f"❌ 중복 chunk_id 수: {len(dup_ids)}")
print(f"📏 평균 길이: {avg_len}자")
print(f"📏 최대 길이: {max_len} / 최소 길이: {min_len}\n")

print(f"📚 조문 형식 만족 청크 수: {len(valid_titles)}")
if invalid_titles:
    print(f"⚠️ 형식 불만족 청크 예시: {invalid_titles[:3]}")
else:
    print("✅ 모든 청크가 조문 형식을 만족합니다.")


✅ 총 청크 수: 42
❌ 중복 chunk_id 수: 0
📏 평균 길이: 38자
📏 최대 길이: 515 / 최소 길이: 8

📚 조문 형식 만족 청크 수: 42
✅ 모든 청크가 조문 형식을 만족합니다.


In [18]:
# 4. 청크 보완작업 ( 슬라이딩 윈도우 방식 ) 

import json

# 1. 기존 청크 로드
with open("dm_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# 2. 슬라이딩 윈도우 병합
WINDOW_SIZE = 3
STRIDE = 1
windowed_chunks = []

for i in range(0, len(chunks) - WINDOW_SIZE + 1, STRIDE):
    group = chunks[i:i + WINDOW_SIZE]
    combined_text = "\n".join([c["text"] for c in group])
    section = group[0]["metadata"]["section"]  
    chunk_id = f"{section}_win_{i+1}"

    windowed_chunks.append({
        "text": combined_text,
        "metadata": {
            "section": section,
            "chunk_id": chunk_id
        }
    })

# 3. 저장
with open("dm_chunks_window.json", "w", encoding="utf-8") as f:
    json.dump(windowed_chunks, f, ensure_ascii=False, indent=2)

# 4. 확인
lengths = [len(c["text"]) for c in windowed_chunks]
print(f"✅ 생성 완료: 총 {len(windowed_chunks)}개")
print(f"📏 평균 길이: {sum(lengths)//len(lengths)}자")
print(f"📏 최대 길이: {max(lengths)} / 최소 길이: {min(lengths)}")


✅ 생성 완료: 총 88개
📏 평균 길이: 303자
📏 최대 길이: 527 / 최소 길이: 180


In [ ]:
# 5. 임베딩 ( 청크보완파일 ,kure-v1 , faiss db) 

from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import json
import os

# 🟦 1. 설정
MODEL_NAME = "work1\models\kure_v1"
DEVICE = "cuda"
BATCH_SIZE = 32
DATA_FILE = "dm_chunks_window.json"  
INDEX_DIR = "faiss_win"  

# 🟦 2. JSON 로드
with open(DATA_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

docs = [
    Document(
        page_content=item["text"],
        metadata=item["metadata"]
    ) for item in data
]

# 🟦 3. 임베딩 모델 로드
embedding = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True, "batch_size": BATCH_SIZE}
)

# 🟦 4. 임베딩 + 저장
print("⏳ 임베딩 중...")
vectorstore = FAISS.from_documents(docs, embedding)
vectorstore.save_local(INDEX_DIR)
print(f"✅ 저장 완료: {INDEX_DIR}")




⏳ 임베딩 중...
✅ 저장 완료: faiss_win


In [ ]:
# 6. 평가를 위한 질문지 생성 ( openai gpt-4o )

from openai import OpenAI
import json
from tqdm import tqdm

client = OpenAI(api_key="Key")
                
# 1. 청크 로드
with open("dm_chunks_merged.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

questions = []

# 2. 고품질 질문 유도 프롬프트 생성 + 요청
for chunk in tqdm(chunks):
    prompt = f"""
다음은 사내 규정의 한 조문입니다. 여기에 기반하여 실제 직원들이 물어볼 수 있는 질문을 3개 생성하세요.

[규정 내용]
\"\"\"
{chunk["text"]}
\"\"\"

조건:
- 실제 사용자가 물어볼 법한 질문일 것
- 숫자, 조건문, 예외사항이 있다면 반드시 반영할 것
- 질문은 한국어로, 자연스러운 말투로 작성
- 답변은 반드시 위 텍스트 내에서만 추출 가능해야 함

형식:
[
  {{"question": "...", "answer": "..."}},
  ...
]
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        raw = response.choices[0].message.content
        qa_list = json.loads(raw)
        for qa in qa_list:
            if qa["question"].strip() and qa["answer"].strip():
                questions.append(qa)
    except Exception as e:
        print(f"[⚠️ Error] 청크 ID: {chunk['metadata']['chunk_id']} → {e}")

# 3. 저장
with open("eval_questions_merged.jsonl", "w", encoding="utf-8") as f:
    for q in questions:
        f.write(json.dumps(q, ensure_ascii=False) + "\n")

print(f"✅ 저장 완료: 총 {len(questions)}개 문항 → eval_questions_merged.jsonl")


100%|██████████| 44/44 [01:54<00:00,  2.60s/it]

✅ 저장 완료: 총 132개 문항 → eval_questions.jsonl


In [37]:
# 질문지 결과 확인

print("📤 GPT 응답 내용 확인:")
print(repr(response.choices[0].message.content))


📤 GPT 응답 내용 확인:
'[\n  {"question": "서약서를 제출하지 않으면 어떻게 되나요?", "answer": "모든 임직원은 본 강령을 숙지하고 서약서를 제출해야 하며, 위반 시 책임을 진다."},\n  {"question": "서약서는 얼마나 오랫동안 보관되나요?", "answer": "서약서는 인사기록과 함께 3년간 보관된다."},\n  {"question": "서약서를 제출해야 하는 사람은 누구인가요?", "answer": "모든 임직원은 본 강령을 숙지하고 서약서를 제출해야 한다."}\n]'
